With pkcomposite -cr mean i can maybe do it, although this is ment to calculate the mean between 2 layers. 
so maybe not. Lateron I can use it to combine all my maps tho, and include weights. 

Here we take the 10m CLC map for Austria turn it into forest/nonforest and downsample it to the 100m map

In [2]:
!ls in_data/CLC/CLC_2018_10m/Data/

AUT.tif
AUT.tif.aux.xml
CLMS_CLCplus_RASTER_2018_010m_eu_03035_V1_1.tif
CLMS_CLCplus_RASTER_2018_010m_eu_03035_V1_1.tif.aux.xml
CLMS_CLCplus_RASTER_2018_010m_eu_03035_V1_1.tif.clr
CLMS_CLCplus_RASTER_2018_010m_eu_03035_V1_1.tif.clr.txt
CLMS_CLCplus_RASTER_2018_010m_eu_03035_V1_1.tif.ovr
CLMS_CLCplus_RASTER_2018_010m_eu_03035_V1_1.tif.vat.cpg
CLMS_CLCplus_RASTER_2018_010m_eu_03035_V1_1.tif.vat.dbf
CLMS_CLCplus_RASTER_2018_010m_eu_03035_V1_1.tif.vat.dbf.xml
French_DOMs
makeAUT.sh
makeAUT.sh~


In [1]:
!gdalinfo in_data/CLC/CLC_2018_10m/Data/AUT.tif

Driver: GTiff/GeoTIFF
Files: in_data/CLC/CLC_2018_10m/Data/AUT.tif
       in_data/CLC/CLC_2018_10m/Data/AUT.tif.aux.xml
Size is 26917, 29388
Coordinate System is:
PROJCRS["ETRS89-extended / LAEA Europe",
    BASEGEOGCRS["ETRS89",
        ENSEMBLE["European Terrestrial Reference System 1989 ensemble",
            MEMBER["European Terrestrial Reference Frame 1989"],
            MEMBER["European Terrestrial Reference Frame 1990"],
            MEMBER["European Terrestrial Reference Frame 1991"],
            MEMBER["European Terrestrial Reference Frame 1992"],
            MEMBER["European Terrestrial Reference Frame 1993"],
            MEMBER["European Terrestrial Reference Frame 1994"],
            MEMBER["European Terrestrial Reference Frame 1996"],
            MEMBER["European Terrestrial Reference Frame 1997"],
            MEMBER["European Terrestrial Reference Frame 2000"],
            MEMBER["European Terrestrial Reference Frame 2005"],
            MEMBER["European Terrestrial Referen

I can maybe even do it with gdaltranslate after I recode all to forest/nonforest
with the option -r average
and the output resolution set to how I want it. 

    recode to forest/nonforest/NA
        First lets create a list of forest classes

In [40]:
%%bash
for((i=1;i<=11;++i));do
  if(($i >=2 && $i <=4));then
    echo "$i 1"
  else
    echo "$i 0";fi
done > /tmp/recode4bash.txt 

echo "254 48" >> /tmp/recode4bash.txt 
echo "255 48" >> /tmp/recode4bash.txt 

In [41]:
!head /tmp/recode4bash.txt 

1 0
2 1
3 1
4 1
5 0
6 0
7 0
8 0
9 0
10 0


In [93]:
%%bash
start=`date +%s`
pkreclass -co COMPRESS=DEFLATE -co ZLEVEL=9  \
-code /tmp/recode4bash.txt                   \
-nodata 48                                   \
-ot Float64                                  \
-i in_data/CLC/CLC_2018_10m/Data/AUT.tif     \
-o out_data/CLC10m/AUT_recode.tif            \

end=`date +%s`

expr $end - $start

0...10...20...30...40...50...60...70...80...90...100 - done.
361


Get the extent and resolution from the 100m map and use that to calculate the mean

In [95]:
!gdalinfo out_data/CLC/CLC_recode01.tif | grep Pixel

Pixel Size = (100.000000000000000,-100.000000000000000)


In [96]:
!gdalinfo out_data/CLC/CLC_recode01.tif | grep Origin
!gdalinfo out_data/CLC10m/AUT_recode.tif| grep Origin
!gdalinfo in_data/CLC/CLC_2018_10m/Data/CLMS_CLCplus_RASTER_2018_010m_eu_03035_V1_1.tif| grep Origin


Origin = (900000.000000000000000,5500000.000000000000000)
Origin = (4585465.000000000000000,2889034.000000000000000)
Warning 1: The definition of projected CRS EPSG:3035 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.
Origin = (900000.000000000000000,5500000.000000000000000)


In [97]:
%%bash
start=`date +%s`
gdal_translate -ot Float64                                       \
-tr 100 100                                                      \
-r average                                                       \
out_data/CLC10m/AUT_recode.tif out_data/CLC10m/AUT_100m_mean.tif \

end=`date +%s`

expr $end - $start

Input file size is 26917, 29388
0...10...20...30...40...50...60...70...80...90...100 - done.
13
